<a href="https://colab.research.google.com/github/bbi-yggy/keras-jumpnet/blob/main/jumpnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing JumpNet and training with CIFAR-10
Deep Learning for Computer Vision (Cohort #5)  
Yossarian King / Blackbird Interactive / October 2020

The JumpNet implementation is based on the description from [keras-idiomatic-programmer/zoo/jumpnet](https://github.com/GoogleCloudPlatform/keras-idiomatic-programmer/tree/master/zoo/jumpnet).

The implementation is in the idiomatic style - stem > learner > classifier.
The learner consists of groups, each composed of blocks, as illustrated here:

![JumpNet diagram](https://github.com/GoogleCloudPlatform/keras-idiomatic-programmer/blob/master/zoo/jumpnet/macro.jpg?raw=true)

For clarity of implementation, the code has been built as a class with a fluent API, allowing all model metaparameters to be fed into fluent method calls as named parameters. This makes the model structure explicit and the parameters clear, and enables flexible use to build different models in the JumpNet style.

Metaparameters used in this notebook precisely mimic the model built by [jumpnet_c.py](https://github.com/GoogleCloudPlatform/keras-idiomatic-programmer/tree/master/zoo/jumpnet/jumpnet_c.py) (see the comment "Example of JumpNet for CIFAR-10").

The model is trained on [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html).
